In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.datasets import CIFAR10
import matplotlib.pyplot as plt
import pandas as pd
import time
from torch.utils.tensorboard import SummaryWriter


    Creates a Convolutional Neural Network (CNN) with customizable kernel size and pooling type.

    Parameters:
        kernel_size (int): The size of the kernel to use in convolutional layers (e.g., 3, 5, 7).
        pooling_type (nn.Module): The pooling layer type (e.g., nn.MaxPool2d or nn.AvgPool2d).

    Returns:
        nn.Module: A CNN model ready for training and evaluation.
    

In [14]:
# Define the CNN Model
def create_cnn(kernel_size, pooling_type):
    class CNN(nn.Module):
        def __init__(self):
            super(CNN, self).__init__()
            self.conv1 = nn.Conv2d(3, 32, kernel_size=kernel_size, padding=kernel_size//2)
            self.pool1 = pooling_type(kernel_size=2, stride=2)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=kernel_size//2)
            self.pool2 = pooling_type(kernel_size=2, stride=2)
            self.fc1 = nn.Linear(64 * 8 * 8, 512)
            self.fc2 = nn.Linear(512, 10)

        def forward(self, x):
            x = self.pool1(torch.relu(self.conv1(x)))
            x = self.pool2(torch.relu(self.conv2(x)))
            x = x.view(-1, 64 * 8 * 8)
            x = torch.relu(self.fc1(x))
            x = self.fc2(x)
            return x

    return CNN()


    Train and evaluate a given CNN model.

    Parameters:
        model (nn.Module): The CNN model to train and evaluate.
        optimizer (torch.optim.Optimizer): The optimizer used for training (e.g., SGD, Adam).
        scheduler (torch.optim.lr_scheduler._LRScheduler): Learning rate scheduler to adjust learning rates.
        train_loader (DataLoader): DataLoader for training data.
        test_loader (DataLoader): DataLoader for testing data.
        criterion (nn.Module): Loss function (e.g., CrossEntropyLoss).
        device (torch.device): Device to use for computation (e.g., 'cuda' or 'cpu').
        epochs (int): Number of training epochs.
        config (str): Configuration details for this experiment.
        results (list): List to store results of experiments.
        patience (int): Number of epochs with no improvement to stop training early.

    Logs training loss and test accuracy for each epoch using TensorBoard and stores results.
    

In [15]:
# Define Training Function
def train_and_evaluate(model, optimizer, scheduler, train_loader, test_loader, criterion, device, epochs, config, results, patience=10):
    writer = SummaryWriter()
    model.to(device)
    start_time = time.time()
    best_accuracy = 0.0
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()  # Enable training mode for the model
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        scheduler.step(train_loss / len(train_loader))

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

    elapsed_time = time.time() - start_time
    results.append({
        'config': config,
        'epochs': epoch + 1,
        'best_accuracy': best_accuracy,
        'elapsed_time': elapsed_time
    })
    writer.close()

In [16]:
# Load CIFAR-10 Dataset
# Explanation: CIFAR-10 images are scaled to [0, 1] with ToTensor, then normalized to zero mean and unit variance.
# The mean and standard deviation values (0.5) are standard for datasets with RGB channels.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=12, pin_memory=True, prefetch_factor=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=12, pin_memory=True, prefetch_factor=2)

# Results storage
results = []

Files already downloaded and verified
Files already downloaded and verified


In [17]:
# Experiment for Kernel Size
kernel_sizes = [3, 5, 7]
for kernel_size in kernel_sizes:
    config = f"KernelSize={kernel_size}"
    model = create_cnn(kernel_size, nn.MaxPool2d)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    scheduler = ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)
    train_and_evaluate(model, optimizer, scheduler, train_loader, test_loader, criterion, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), 50, config, results)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/50, Loss: 2.1824, Accuracy: 29.84%
Epoch 2/50, Loss: 1.8533, Accuracy: 38.78%
Epoch 3/50, Loss: 1.6878, Accuracy: 43.39%
Epoch 4/50, Loss: 1.5636, Accuracy: 46.96%
Epoch 5/50, Loss: 1.4678, Accuracy: 49.05%
Epoch 6/50, Loss: 1.4020, Accuracy: 49.61%
Epoch 7/50, Loss: 1.3457, Accuracy: 49.71%
Epoch 8/50, Loss: 1.2998, Accuracy: 52.52%
Epoch 9/50, Loss: 1.2559, Accuracy: 54.24%
Epoch 10/50, Loss: 1.2132, Accuracy: 57.12%
Epoch 11/50, Loss: 1.1790, Accuracy: 57.13%
Epoch 12/50, Loss: 1.1429, Accuracy: 58.32%
Epoch 13/50, Loss: 1.1113, Accuracy: 59.17%
Epoch 14/50, Loss: 1.0764, Accuracy: 60.66%
Epoch 15/50, Loss: 1.0513, Accuracy: 60.69%
Epoch 16/50, Loss: 1.0174, Accuracy: 62.25%
Epoch 17/50, Loss: 0.9923, Accuracy: 63.07%
Epoch 18/50, Loss: 0.9627, Accuracy: 63.16%
Epoch 19/50, Loss: 0.9358, Accuracy: 63.24%
Epoch 20/50, Loss: 0.9103, Accuracy: 63.83%
Epoch 21/50, Loss: 0.8842, Accuracy: 63.40%
Epoch 22/50, Loss: 0.8604, Accuracy: 64.46%
Epoch 23/50, Loss: 0.8357, Accuracy: 65.5

In [18]:
# Experiment for Pooling Types
pooling_types = [nn.MaxPool2d, nn.AvgPool2d]
for pooling_type in pooling_types:
    config = f"PoolingType={pooling_type.__name__}"
    model = create_cnn(3, pooling_type)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    scheduler = ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)
    train_and_evaluate(model, optimizer, scheduler, train_loader, test_loader, criterion, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), 50, config, results)

Epoch 1/50, Loss: 2.1759, Accuracy: 29.59%
Epoch 2/50, Loss: 1.8841, Accuracy: 37.44%
Epoch 3/50, Loss: 1.7226, Accuracy: 41.33%
Epoch 4/50, Loss: 1.6171, Accuracy: 45.24%
Epoch 5/50, Loss: 1.5202, Accuracy: 47.62%
Epoch 6/50, Loss: 1.4350, Accuracy: 49.53%
Epoch 7/50, Loss: 1.3680, Accuracy: 50.46%
Epoch 8/50, Loss: 1.3160, Accuracy: 52.57%
Epoch 9/50, Loss: 1.2643, Accuracy: 54.10%
Epoch 10/50, Loss: 1.2219, Accuracy: 51.18%
Epoch 11/50, Loss: 1.1898, Accuracy: 57.38%
Epoch 12/50, Loss: 1.1513, Accuracy: 58.37%
Epoch 13/50, Loss: 1.1203, Accuracy: 59.86%
Epoch 14/50, Loss: 1.0924, Accuracy: 60.69%
Epoch 15/50, Loss: 1.0607, Accuracy: 60.67%
Epoch 16/50, Loss: 1.0326, Accuracy: 60.25%
Epoch 17/50, Loss: 1.0036, Accuracy: 61.74%
Epoch 18/50, Loss: 0.9804, Accuracy: 61.17%
Epoch 19/50, Loss: 0.9511, Accuracy: 62.94%
Epoch 20/50, Loss: 0.9272, Accuracy: 62.13%
Epoch 21/50, Loss: 0.9019, Accuracy: 65.89%
Epoch 22/50, Loss: 0.8798, Accuracy: 62.65%
Epoch 23/50, Loss: 0.8549, Accuracy: 65.9

In [19]:
# Experiment for Epochs
epoch_list = [5, 50, 100, 250, 350]
for epochs in epoch_list:
    config = f"Epochs={epochs}"
    model = create_cnn(3, nn.MaxPool2d)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    scheduler = ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)
    train_and_evaluate(model, optimizer, scheduler, train_loader, test_loader, criterion, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), epochs, config, results)

Epoch 1/5, Loss: 2.1955, Accuracy: 29.05%
Epoch 2/5, Loss: 1.8907, Accuracy: 38.08%
Epoch 3/5, Loss: 1.6992, Accuracy: 42.44%
Epoch 4/5, Loss: 1.5654, Accuracy: 46.15%
Epoch 5/5, Loss: 1.4657, Accuracy: 47.54%
Epoch 1/50, Loss: 2.1730, Accuracy: 30.67%
Epoch 2/50, Loss: 1.8518, Accuracy: 38.24%
Epoch 3/50, Loss: 1.6994, Accuracy: 41.79%
Epoch 4/50, Loss: 1.5979, Accuracy: 46.16%
Epoch 5/50, Loss: 1.5036, Accuracy: 48.78%
Epoch 6/50, Loss: 1.4246, Accuracy: 49.67%
Epoch 7/50, Loss: 1.3661, Accuracy: 52.01%
Epoch 8/50, Loss: 1.3137, Accuracy: 54.17%
Epoch 9/50, Loss: 1.2698, Accuracy: 55.50%
Epoch 10/50, Loss: 1.2300, Accuracy: 52.92%
Epoch 11/50, Loss: 1.1915, Accuracy: 58.05%
Epoch 12/50, Loss: 1.1566, Accuracy: 57.87%
Epoch 13/50, Loss: 1.1269, Accuracy: 60.17%
Epoch 14/50, Loss: 1.0983, Accuracy: 60.49%
Epoch 15/50, Loss: 1.0702, Accuracy: 60.21%
Epoch 16/50, Loss: 1.0387, Accuracy: 60.96%
Epoch 17/50, Loss: 1.0107, Accuracy: 62.01%
Epoch 18/50, Loss: 0.9855, Accuracy: 61.75%
Epoch 1

In [20]:
# Experiment for Optimizers
optimizers = ['SGD', 'RMSprop', 'Adam']
for optimizer_name in optimizers:
    config = f"Optimizer={optimizer_name}"
    model = create_cnn(3, nn.MaxPool2d)
    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=0.01)
    scheduler = ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)
    train_and_evaluate(model, optimizer, scheduler, train_loader, test_loader, criterion, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), 50, config, results)

Epoch 1/50, Loss: 2.1481, Accuracy: 32.10%
Epoch 2/50, Loss: 1.8331, Accuracy: 38.94%
Epoch 3/50, Loss: 1.6550, Accuracy: 44.44%
Epoch 4/50, Loss: 1.5370, Accuracy: 46.88%
Epoch 5/50, Loss: 1.4587, Accuracy: 50.13%
Epoch 6/50, Loss: 1.3947, Accuracy: 49.78%
Epoch 7/50, Loss: 1.3481, Accuracy: 50.71%
Epoch 8/50, Loss: 1.3046, Accuracy: 53.94%
Epoch 9/50, Loss: 1.2634, Accuracy: 55.41%
Epoch 10/50, Loss: 1.2262, Accuracy: 55.03%
Epoch 11/50, Loss: 1.1938, Accuracy: 56.48%
Epoch 12/50, Loss: 1.1612, Accuracy: 55.39%
Epoch 13/50, Loss: 1.1330, Accuracy: 58.73%
Epoch 14/50, Loss: 1.1014, Accuracy: 59.21%
Epoch 15/50, Loss: 1.0688, Accuracy: 59.09%
Epoch 16/50, Loss: 1.0417, Accuracy: 60.76%
Epoch 17/50, Loss: 1.0132, Accuracy: 61.90%
Epoch 18/50, Loss: 0.9843, Accuracy: 62.74%
Epoch 19/50, Loss: 0.9566, Accuracy: 63.47%
Epoch 20/50, Loss: 0.9310, Accuracy: 61.62%
Epoch 21/50, Loss: 0.9060, Accuracy: 62.21%
Epoch 22/50, Loss: 0.8775, Accuracy: 64.97%
Epoch 23/50, Loss: 0.8555, Accuracy: 65.1

In [21]:
# Display results
results_df = pd.DataFrame(results)
print(results_df)

                   config  epochs  best_accuracy  elapsed_time
0            KernelSize=3      50          70.61    168.694649
1            KernelSize=5      50          71.56    169.563034
2            KernelSize=7      50          72.45    175.105135
3   PoolingType=MaxPool2d      50          70.63    173.122606
4   PoolingType=AvgPool2d      50          64.36    174.971585
5                Epochs=5       5          47.54     17.667847
6               Epochs=50      50          70.69    178.248035
7              Epochs=100      53          71.39    192.121496
8              Epochs=250      73          70.52    266.859167
9              Epochs=350      52          70.70    190.289392
10          Optimizer=SGD      50          69.44    184.739929
11      Optimizer=RMSprop      25          60.00     93.816151
12         Optimizer=Adam      17          52.60     64.482401
